# Multi-task SELD


In [1]:
import torch
from torchinfo import summary

from datasets import create_dataloaders
from multi_task import MultiTaskSELD, train_model

SAMPLE_RATE = 24000
FRAME_LENGTH = SAMPLE_RATE // 10
MAX_EVENTS = 5
NUM_CLASSES = 13
BATCH_SIZE = 256

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


## Model

In [2]:
model_args = {
    'backbone': 'seldnet',
    'num_classes': NUM_CLASSES,
    'num_events': MAX_EVENTS,
    'input_dim': 7,
    'hidden_dim': 64,
    'dropout': 0.05,
    'rnn_layers': 2,
    'mhsa_layers': 2,
}
train_dataloader, _ = create_dataloaders(BATCH_SIZE)
model = MultiTaskSELD(**model_args)
features, *_ = next(iter(train_dataloader))
summary(model, input_data=[features])

Layer (type:depth-idx)                   Output Shape              Param #
MultiTaskSELD                            [256, 50, 13, 4]          --
├─SELDNetBackbone: 1-1                   [256, 50, 128]            --
│    └─Sequential: 2-1                   [256, 64, 2, 50]          --
│    │    └─ConvBlock: 3-1               [256, 64, 16, 50]         4,224
│    │    └─ConvBlock: 3-2               [256, 64, 4, 50]          37,056
│    │    └─ConvBlock: 3-3               [256, 64, 2, 50]          37,056
│    └─GRU: 2-2                          [256, 50, 256]            494,592
│    └─ModuleList: 2-5                   --                        (recursive)
│    │    └─MultiheadAttention: 3-4      [256, 50, 128]            66,048
│    └─ModuleList: 2-6                   --                        (recursive)
│    │    └─LayerNorm: 3-5               [256, 50, 128]            256
│    └─ModuleList: 2-5                   --                        (recursive)
│    │    └─MultiheadAttention: 3-6  

## Evaluation

In [3]:
model_args = {
    'backbone': 'seldnet',
    'num_classes': NUM_CLASSES,
    'num_events': MAX_EVENTS,
    'input_dim': 7,
    'hidden_dim': 64,
    'dropout': 0.05,
    'rnn_layers': 2,
    'mhsa_layers': 2,
}
train_dataloader, test_dataloader = create_dataloaders(BATCH_SIZE)
model = train_model(
    model_args,
    train_dataloader,
    test_dataloader,
    epochs=250,
    device=device,
    sde_weight=0.0,
)

Epoch 10/250: 100%|██████████| 12/12 [00:19<00:00,  1.59s/it, loss=0.0165, test_loss=0.0291]


Macro: ER=0.89, F=0.04, LE=134.01, LR=0.08
Micro: ER=0.89, F=0.10, LE=31.64, LR=0.17


Epoch 20/250: 100%|██████████| 12/12 [00:21<00:00,  1.78s/it, loss=0.0102, test_loss=0.0264]


Macro: ER=0.79, F=0.07, LE=134.17, LR=0.16
Micro: ER=0.79, F=0.19, LE=30.97, LR=0.39


Epoch 30/250: 100%|██████████| 12/12 [00:21<00:00,  1.80s/it, loss=0.00823, test_loss=0.0259]


...

Macro: ER=0.74, F=0.15, LE=46.55, LR=0.30
Micro: ER=0.74, F=0.28, LE=30.40, LR=0.50


Epoch 240/250: 100%|██████████| 12/12 [00:21<00:00,  1.79s/it, loss=0.002, test_loss=0.0273]


Macro: ER=0.76, F=0.14, LE=48.73, LR=0.33
Micro: ER=0.76, F=0.29, LE=30.60, LR=0.56


Epoch 250/250: 100%|██████████| 12/12 [00:21<00:00,  1.81s/it, loss=0.00205, test_loss=0.0289]

Macro: ER=0.78, F=0.15, LE=47.33, LR=0.33
Micro: ER=0.78, F=0.27, LE=31.75, LR=0.54
Micro @ epoch 189: ER=0.76, F=0.17, LE=46.21, LR=0.33
Macro @ epoch 189: ER=0.76, F=0.29, LE=30.87, LR=0.57


### Data normalization


In [3]:
model_args = {
    'backbone': 'seldnet',
    'num_classes': NUM_CLASSES,
    'num_events': MAX_EVENTS,
    'input_dim': 7,
    'hidden_dim': 64,
    'dropout': 0.05,
    'rnn_layers': 2,
    'mhsa_layers': 2,
}
train_dataloader, test_dataloader = create_dataloaders(BATCH_SIZE, normalized=True)
model = train_model(
    model_args,
    train_dataloader,
    test_dataloader,
    epochs=250,
    device=device,
    sde_weight=0.0,
)

Epoch 10/250: 100%|██████████| 12/12 [00:19<00:00,  1.60s/it, loss=0.0149, test_loss=0.0265]


Macro: ER=0.85, F=0.07, LE=133.06, LR=0.09
Micro: ER=0.85, F=0.16, LE=27.56, LR=0.20


Epoch 20/250: 100%|██████████| 12/12 [00:20<00:00,  1.74s/it, loss=0.0098, test_loss=0.0267]


Macro: ER=0.79, F=0.08, LE=134.14, LR=0.16
Micro: ER=0.79, F=0.22, LE=31.30, LR=0.39


Epoch 30/250: 100%|██████████| 12/12 [00:20<00:00,  1.72s/it, loss=0.00779, test_loss=0.0253]


...

Macro: ER=0.76, F=0.18, LE=51.75, LR=0.32
Micro: ER=0.76, F=0.31, LE=26.68, LR=0.51


Epoch 240/250: 100%|██████████| 12/12 [00:23<00:00,  1.95s/it, loss=0.00189, test_loss=0.0275]


Macro: ER=0.77, F=0.18, LE=52.17, LR=0.32
Micro: ER=0.77, F=0.31, LE=27.34, LR=0.53


Epoch 250/250: 100%|██████████| 12/12 [00:24<00:00,  2.01s/it, loss=0.00177, test_loss=0.0277]

Macro: ER=0.77, F=0.16, LE=52.54, LR=0.30
Micro: ER=0.77, F=0.30, LE=27.86, LR=0.52
Micro @ epoch 221: ER=0.76, F=0.18, LE=52.33, LR=0.33
Macro @ epoch 221: ER=0.76, F=0.31, LE=27.10, LR=0.53


### Data augmentation


In [ ]:
model_args = {
    'backbone': 'seldnet',
    'num_classes': NUM_CLASSES,
    'num_events': MAX_EVENTS,
    'input_dim': 7,
    'hidden_dim': 64,
    'dropout': 0.05,
    'rnn_layers': 2,
    'mhsa_layers': 2,
}
train_dataloader, test_dataloader = create_dataloaders(
    BATCH_SIZE, augments=[0, 1, 2, 3, 4, 5, 6, 7]
)
model = train_model(
    model_args,
    train_dataloader,
    test_dataloader,
    epochs=40,
    device=device,
    sde_weight=0.0,
)
torch.save(model.state_dict(), 'data/seldnet_augmented.pth')

Epoch 10/40: 100%|██████████| 95/95 [02:24<00:00,  1.52s/it, loss=0.0078, test_loss=0.0215]


Macro: ER=0.68, F=0.13, LE=120.42, LR=0.21
Micro: ER=0.68, F=0.34, LE=25.36, LR=0.53


Epoch 20/40: 100%|██████████| 95/95 [02:26<00:00,  1.54s/it, loss=0.00591, test_loss=0.021]


Macro: ER=0.66, F=0.15, LE=118.57, LR=0.23
Micro: ER=0.66, F=0.39, LE=21.51, LR=0.56


Epoch 30/40: 100%|██████████| 95/95 [02:22<00:00,  1.50s/it, loss=0.00493, test_loss=0.0212]


Macro: ER=0.65, F=0.16, LE=108.72, LR=0.25
Micro: ER=0.65, F=0.40, LE=24.21, LR=0.60


Epoch 40/40: 100%|██████████| 95/95 [02:23<00:00,  1.51s/it, loss=0.00412, test_loss=0.0201]

Macro: ER=0.64, F=0.19, LE=49.90, LR=0.32
Micro: ER=0.64, F=0.41, LE=22.30, LR=0.62
Micro @ epoch 37: ER=0.62, F=0.21, LE=48.73, LR=0.30
Macro @ epoch 37: ER=0.62, F=0.44, LE=19.54, LR=0.58


### Distance estimation


In [6]:
model_args = {
    'backbone': 'seldnet',
    'num_classes': NUM_CLASSES,
    'num_events': MAX_EVENTS,
    'input_dim': 7,
    'hidden_dim': 64,
    'dropout': 0.05,
    'rnn_layers': 2,
    'mhsa_layers': 2,
}
train_dataloader, test_dataloader = create_dataloaders(BATCH_SIZE)
model = train_model(
    model_args,
    train_dataloader,
    test_dataloader,
    epochs=250,
    device=device,
    sde_weight=0.5,
)

Epoch 10/250: 100%|██████████| 12/12 [00:20<00:00,  1.68s/it, loss=0.0582, test_loss=0.138]


Macro: ER=0.87, F=0.03, LE=136.22, LR=0.13
Micro: ER=0.87, F=0.09, LE=38.82, LR=0.26


Epoch 20/250: 100%|██████████| 12/12 [00:21<00:00,  1.75s/it, loss=0.0447, test_loss=0.128]


Macro: ER=0.81, F=0.06, LE=136.30, LR=0.18
Micro: ER=0.81, F=0.16, LE=38.13, LR=0.41


Epoch 30/250: 100%|██████████| 12/12 [00:22<00:00,  1.85s/it, loss=0.0409, test_loss=0.133]


...

Macro: ER=0.79, F=0.09, LE=94.39, LR=0.24
Micro: ER=0.79, F=0.25, LE=34.63, LR=0.55


Epoch 240/250: 100%|██████████| 12/12 [00:22<00:00,  1.85s/it, loss=0.0143, test_loss=0.132]


Macro: ER=0.79, F=0.09, LE=93.16, LR=0.24
Micro: ER=0.79, F=0.25, LE=35.26, LR=0.56


Epoch 250/250: 100%|██████████| 12/12 [00:22<00:00,  1.85s/it, loss=0.0142, test_loss=0.131]

Macro: ER=0.80, F=0.10, LE=94.94, LR=0.25
Micro: ER=0.80, F=0.26, LE=35.13, LR=0.56
Micro @ epoch 246: ER=0.79, F=0.11, LE=93.29, LR=0.24
Macro @ epoch 246: ER=0.79, F=0.26, LE=35.12, LR=0.55
